# Acesso genérico a atributos

Dois métodos poderosos são `__getattr__` e `__setattr__`. Eles permitem lidar de forma especial com acessos a atributos de objetos.

O método `__getattr__` é chamado sempre que se tenta _acessar para leitura_ um atributo _que não foi definido_ para o objeto.

O método `__setattr__` é chamado sempre que se deseja _alterar o valor_ de um atributo, _seja ele definido ou não_. Neste caso, como ele sempre é chamado, **deve-se ter cuidado com o acesso a atributos dentro do próprio `__setattr__`**: os acessos devem ser feitos através do dicionário do objeto, `__dict__`, que contém os pares nome/valor dos atributos.

In [ ]:
class Empty:
    def __init__(self):
        self.a = 1
        self.b = 2
        
    def __getattr__(self, name):
        print(f'Accessing new attribute {name}, of object {self}')
        self.__dict__[name] = 0
        return 0
        
    def __setattr__(self, name, val):
        print(f'Setting value of attribute {name} of object {self} to {val}')
        self.__dict__[name] = val

In [ ]:
v = Empty()

Note como os acessos aos atributos `a` e `b` no construtor passam pelo `__setattr__`.

O código abaixo mostra que o atributo `__dict__` contém os pares chave/valor correspondentes aos atributos criados para o objeto pelo `__init__`.

In [ ]:
v

In [ ]:
v.__dict__

O acesso a esses atributos existente para leitura é normal:

In [ ]:
v.b

Mas se acessarmos um atributo inexistente, o `__getattr__` será executado.

In [ ]:
v.c

Agora o atributo `c` já existe, portanto novos acessos de leitura não passam por `__getattr__`.

In [ ]:
v.__dict__

In [ ]:
v.c

Agora se tentarmos mudar o valor de um atributo existente ou não, será executado o código de `__setattr__`.

In [ ]:
v.a = 5

In [ ]:
v.d = 3

In [ ]:
v.__dict__

Um outro método denominado `__getattribute__` permite que **todos** os acessos de leitura (mesmo para atributos já existentes) passem por uma chamada a esse método. Veja a nova versão do exemplo abaixo:

In [ ]:
class Empty2:
    def __init__(self):
        self.a = 1
        self.b = 2
        
    def __getattr__(self, name):
        print(f'The attribute {name} is new in object {self}')
        super().__getattribute__('__dict__')[name] = 0
        return 0
        
    def __getattribute__(self, name):
        print(f'Accessing attribute {name}, of object {self}')
        return super().__getattribute__(name)
        
    def __setattr__(self, name, val):
        print(f'Setting value of attribute {name} of object {self} to {val}')
        self.__dict__[name] = val

In [ ]:
e2 = Empty2()

In [ ]:
e2.c

In [ ]:
e2.c

Note como aqui fizemos uso de `super()` para chamar a implementação de acesso a atributo da classe base. Isso deve sempre ser feito quando implementamos `__getattribute__` numa classe, para evitar recursão infinita!

Veja que se por exemplo na implementação de `__getattribute__` ao invés de
```python
return super().__getattribute__(name)
```
tivessemos escrito
```python
return self.__dict__[name]
```
então o acesso a `__dict__` indicado neste último trecho de código provocaria uma chamada a `__getattribute__`, gerando uma recursão que nunca terminaria!

Esses métodos são necessários apenas em situações muito especiais, mas são muito úteis nesses casos!